In [ ]:
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show
output_notebook()

In [ ]:
from app.utils.process_gtimelog import get_work_df, add_processed_columns
processed = add_processed_columns(get_work_df())
processed.head()

In [ ]:
import datetime
today = datetime.date.today()
one_week_ago = today - datetime.timedelta(weeks=1)
two_week_ago = today - datetime.timedelta(weeks=2)
three_week_ago = today - datetime.timedelta(weeks=3)

start = one_week_ago
end = today

one_week = processed[(processed.timestamp >= start) & (processed.timestamp <= end)]
one_week = one_week[one_week.activity != 'start']
one_week['formatted_activity'] = one_week.parent_activity + ' (' + one_week.sub_activity + ')'
one_week['activity_bottom'] = one_week.formatted_activity + ':0.25'
one_week['activity_top'] = one_week.formatted_activity + ':0.75'


one_week.head()

In [ ]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Range1d, FactorRange, DataRange1d, HoverTool, PanTool, WheelZoomTool
source = ColumnDataSource(one_week[['start', 'end', 'activity_top', 'activity_bottom', 'human']])

activities = list(one_week.formatted_activity.unique())
activities = sorted(activities)
n = len(activities)

p = figure(
    x_range=Range1d(start=start, end=end),
    y_range=FactorRange(factors=activities),
    tools='reset', width=600, height=100*n
)
p.quad(left='start', right='end', top='activity_top', bottom='activity_bottom', source=source)
p.add_tools(HoverTool(tooltips='@human hrs'))
p.add_tools(PanTool(dimensions=['width']))
p.add_tools(WheelZoomTool(dimensions=['width']))
show(p)